# Obesity Data Preprocessing

## Import packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Meta data consideration

In [2]:
# Display all columns
pd.set_option("display.max_columns", None)
# Load meta data
dbt = pd.read_csv('../data/diabete_type2/SraRunTable.txt')
dbt.head()

,Run,Age,alignment_software (exp),Assay Type,AssemblyName,AvgSpotLen,Bases,BIOMATERIAL_PROVIDER,BioProject,BioSample,BioSampleModel,Bytes,Center Name,Consent,DATASTORE filetype,DATASTORE provider,DATASTORE region,Experiment,Instrument,Isolate,Library Name,LibraryLayout,LibrarySelection,LibrarySource,Organism,Platform,Population,ReleaseDate,Sample Name,sex,SRA Study,Tissue,PHENOTYPE
0,SRR2969603,55.0,BWA v0.59,WXS,GCA_000001405.13,159,3099467700,"Dept of Genetic Medicine\, Weill Cornell Medic...",PRJNA290484,SAMN04304774,Human,1249295315,WEILL CORNELL,public,"sra,bam","s3,gs","gs.US,s3.us-east-1",SRX1458830,Illumina HiSeq 2000,DGMQ-31059_adult_female_Qatari_blood,DGMQ-31059_WXS,PAIRED,RANDOM,GENOMIC,Homo sapiens,ILLUMINA,Qatari,2016-04-13T00:00:00Z,DGMQ-31059,female,SRP061943,blood,NaN
1,SRR2969604,68.0,BWA v0.59,WXS,GCA_000001405.13,160,3719918160,"Dept of Genetic Medicine\, Weill Cornell Medic...",PRJNA290484,SAMN04304775,Human,1597202431,WEILL CORNELL,public,"sra,bam","s3,gs","s3.us-east-1,gs.US",SRX1458831,Illumina HiSeq 2000,DGMQ-31083_adult_female_Qatari_blood,DGMQ-31083_WXS,PAIRED,RANDOM,GENOMIC,Homo sapiens,ILLUMINA,Qatari,2016-04-13T00:00:00Z,DGMQ-31083,female,SRP061943,blood,NaN
2,SRR2969605,22.0,BWA v0.59,WXS,GCA_000001405.13,166,4276615590,"Dept of Genetic Medicine\, Weill Cornell Medic...",PRJNA290484,SAMN04304773,Human,1887095760,WEILL CORNELL,public,"sra,bam","gs,s3","s3.us-east-1,gs.US",SRX1458832,Illumina HiSeq 2000,DGMQ-32079_adult_female_Qatari_blood,DGMQ-32079_WXS,PAIRED,RANDOM,GENOMIC,Homo sapiens,ILLUMINA,Qatari,2016-04-13T00:00:00Z,DGMQ-32079,female,SRP061943,blood,NaN
3,SRR2969606,53.0,BWA v0.59,WXS,GCA_000001405.13,170,2477404800,"Dept of Genetic Medicine\, Weill Cornell Medic...",PRJNA290484,SAMN04304777,Human,1168524324,WEILL CORNELL,public,"sra,bam","s3,gs","s3.us-east-1,gs.US",SRX1458833,Illumina HiSeq 2000,DGMQ-31199_adult_male_Qatari_blood,DGMQ-31199_WXS,PAIRED,RANDOM,GENOMIC,Homo sapiens,ILLUMINA,Qatari,2016-04-13T00:00:00Z,DGMQ-31199,male,SRP061943,blood,NaN
4,SRR2969607,17.0,BWA v0.59,WXS,GCA_000001405.13,167,2393881110,"Dept of Genetic Medicine\, Weill Cornell Medic...",PRJNA290484,SAMN04304776,Human,1179464394,WEILL CORNELL,public,"sra,bam","gs,s3","s3.us-east-1,gs.US",SRX1458834,Illumina HiSeq 2000,DGMQ-32084_adult_male_Qatari_blood,DGMQ-32084_WXS,PAIRED,RANDOM,GENOMIC,Homo sapiens,ILLUMINA,Qatari,2016-04-13T00:00:00Z,DGMQ-32084,male,SRP061943,blood,NaN


In [3]:
# Extract whole exome samples with phenotypes
dbt.dropna(inplace=True)
dbt = dbt[dbt['Assay Type']=='WXS']
dbt = dbt.loc[:, ['Run', 'Sample Name', 'Age', 'sex', 'PHENOTYPE']]
dbt = dbt.sort_values(by=['Sample Name'])
dbt.head()

,Run,Sample Name,Age,sex,PHENOTYPE
1046,SRR2125237,DGMQ-31004,56.0,female,type 2 diabetes
1047,SRR2125238,DGMQ-31006,55.0,female,type 2 diabetes
763,SRR2130687,DGMQ-31007,58.0,female,type 2 diabetes
770,SRR2130694,DGMQ-31008,51.0,female,type 2 diabetes
761,SRR2130685,DGMQ-31009,56.0,female,type 2 diabetes


In [4]:
print('Sample size: ', dbt.shape[0])

Sample size:  780


In [5]:
# Save sample names
pd.DataFrame({'sample_names':list(dbt['Sample Name'].astype(str))}).to_csv('../data/diabete_type2/name_list.txt', header=False, index=None)

In [6]:
# Count number of diabete and none diabete
phenotype = dbt['PHENOTYPE']
case = 0
control = 0
for x in phenotype:
    if 'type 2 diabetes' in x:
        case += 1
    elif 'control' in x:
        control += 1
print ('control:', control, 'case:', case)

control: 270 case: 510


## Generate data for training

In [7]:
# Read a vcf file
def read_vcf(vcf_path):
    with open(vcf_path, "rt") as ifile:
          for line in ifile:
            if line.startswith("#CHROM"):
                  vcf_names = [x for x in line.split('\t')]
                  break
    ifile.close()
    data = pd.read_csv(vcf_path, comment='#', delim_whitespace=True, header=None, names=vcf_names)
    return data

In [8]:
# Load genotype data after imputation
ori_geno = read_vcf('../data/diabete_type2/biallelic_rsid.vcf')
print(ori_geno.shape)
ori_geno.head()

(11508, 789)


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,DGMQ-31004.variant,DGMQ-31006.variant2,DGMQ-31007.variant3,DGMQ-31008.variant4,DGMQ-31009.variant5,DGMQ-31011.variant6,DGMQ-31013.variant7,DGMQ-31015.variant8,DGMQ-31016.variant9,DGMQ-31018.variant10,DGMQ-31019.variant11,DGMQ-31022.variant12,DGMQ-31024.variant13,DGMQ-31028.variant14,DGMQ-31031.variant15,DGMQ-31033.variant16,DGMQ-31035.variant17,DGMQ-31042.variant18,DGMQ-31051.variant19,DGMQ-31055.variant20,DGMQ-31056.variant21,DGMQ-31057.variant22,DGMQ-31063.variant23,DGMQ-31064.variant24,DGMQ-31065.variant25,DGMQ-31068.variant26,DGMQ-31069.variant27,DGMQ-31071.variant28,DGMQ-31074.variant29,DGMQ-31075.variant30,DGMQ-31081.variant31,DGMQ-31084.variant32,DGMQ-31089.variant33,DGMQ-31091.variant34,DGMQ-31107.variant35,DGMQ-31110.variant36,DGMQ-31112.variant37,DGMQ-31113.variant38,DGMQ-31118.variant39,DGMQ-31137.variant40,DGMQ-31146.variant41,DGMQ-31152.variant42,DGMQ-31154.variant43,DGMQ-31155.variant44,DGMQ-31162.variant45,DGMQ-31166.variant46,DGMQ-31167.variant47,DGMQ-31168.variant48,DGMQ-31170.variant49,DGMQ-31171.variant50,DGMQ-31172.variant51,DGMQ-31173.variant52,DGMQ-31185.variant53,DGMQ-31186.variant54,DGMQ-31190.variant55,DGMQ-31192.variant56,DGMQ-31193.variant57,DGMQ-31198.variant58,DGMQ-31201.variant59,DGMQ-31205.variant60,DGMQ-31206.variant61,DGMQ-31208.variant62,DGMQ-31210.variant63,DGMQ-31212.variant64,DGMQ-31213.variant65,DGMQ-31214.variant66,DGMQ-31224.variant67,DGMQ-31229.variant68,DGMQ-31234.variant69,DGMQ-31236.variant70,DGMQ-31252.variant71,DGMQ-31264.variant72,DGMQ-31268.variant73,DGMQ-31275.variant74,DGMQ-31281.variant75,DGMQ-31283.variant76,DGMQ-31285.variant77,DGMQ-31290.variant78,DGMQ-31291.variant79,DGMQ-31302.variant80,DGMQ-31303.variant81,DGMQ-31307.variant82,DGMQ-31309.variant83,DGMQ-31314.variant84,DGMQ-31315.variant85,DGMQ-31321.variant86,DGMQ-31331.variant87,DGMQ-31332.variant88,DGMQ-31342.variant89,DGMQ-31345.variant90,DGMQ-31348.variant91,DGMQ-31357.variant92,DGMQ-31358.variant93,DGMQ-31359.variant94,DGMQ-31360.variant95,DGMQ-31372.variant96,DGMQ-31377.variant97,DGMQ-31382.variant98,DGMQ-31384.variant99,DGMQ-31394.variant100,DGMQ-31395.variant101,DGMQ-31397.variant102,DGMQ-31398.variant103,DGMQ-31407.variant104,DGMQ-31408.variant105,DGMQ-31411.variant106,DGMQ-31412.variant107,DGMQ-31413.variant108,DGMQ-31414.variant109,DGMQ-31417.variant110,DGMQ-31418.variant111,DGMQ-31420.variant112,DGMQ-31424.variant113,DGMQ-31429.variant114,DGMQ-31432.variant115,DGMQ-31436.variant116,DGMQ-31438.variant117,DGMQ-31443.variant118,DGMQ-31446.variant119,DGMQ-31447.variant120,DGMQ-31448.variant121,DGMQ-31450.variant122,DGMQ-31453.variant123,DGMQ-31456.variant124,DGMQ-31458.variant125,DGMQ-31459.variant126,DGMQ-31464.variant127,DGMQ-31465.variant128,DGMQ-31466.variant129,DGMQ-31471.variant130,DGMQ-31472.variant131,DGMQ-31475.variant132,DGMQ-31476.variant133,DGMQ-31478.variant134,DGMQ-31479.variant135,DGMQ-31480.variant136,DGMQ-31481.variant137,DGMQ-31484.variant138,DGMQ-31486.variant139,DGMQ-31487.variant140,DGMQ-31494.variant141,DGMQ-31500.variant142,DGMQ-31501.variant143,DGMQ-31502.variant144,DGMQ-31504.variant145,DGMQ-31506.variant146,DGMQ-31512.variant147,DGMQ-31519.variant148,DGMQ-31520.variant149,DGMQ-31524.variant150,DGMQ-31527.variant151,DGMQ-31528.variant152,DGMQ-31530.variant153,DGMQ-31532.variant154,DGMQ-31533.variant155,DGMQ-31534.variant156,DGMQ-31536.variant157,DGMQ-31538.variant158,DGMQ-31547.variant159,DGMQ-31548.variant160,DGMQ-31551.variant161,DGMQ-31553.variant162,DGMQ-31557.variant163,DGMQ-31558.variant164,DGMQ-31561.variant165,DGMQ-31562.variant166,DGMQ-31570.variant167,DGMQ-31576.variant168,DGMQ-31578.variant169,DGMQ-31584.variant170,DGMQ-31585.variant171,DGMQ-31588.variant172,DGMQ-31590.variant173,DGMQ-31593.variant174,DGMQ-31596.variant175,DGMQ-31604.variant176,DGMQ-31616.variant177,DGMQ-31618.variant178,DGMQ-31619.variant179,DGMQ-31620.variant180,DGMQ-31624.variant181,DGMQ-31628.variant182,DGMQ-31629.variant183,DGMQ-31631.variant184,DGMQ-31638.var

In [9]:
# Change sample names 
col = ori_geno.columns
new_col = []
for x in col:
    new_col.append(x.split('.')[0])
geno = pd.DataFrame(data = ori_geno.to_numpy(), columns = new_col)
# Split ID 
id = geno['ID']
new_id = []
for i in id:
    e = i.split(';')
    if len(e) > 1:
        new_id.append(e[1])
    else:
        new_id.append(e[0])
# Change ID
geno['ID'] = new_id
geno = geno.set_index('ID')
geno = geno.iloc[:, 8::]
geno = geno.T

In [10]:
# Encode bi-allelic genotypes by 0, 1, 2
def get_genotype(data):
    # Encode genotype to 0, 1, 2
    df = data.replace(['0|0', '0|1', '1|0', '1|1'], [0, 1, 1, 2])
    return (df)
# Data including encoded genotypes
geno = get_genotype(geno)

In [11]:
# Collect sex data
s = list(dbt['sex'])
se = []
for x in s:
    if x == 'male':
        se.append(0)
    elif x == 'female':
        se.append(1)
    else:
        se.append(-9)
# Collect phenotype data
pn = list(dbt['PHENOTYPE'])
pheno = []
for x in pn:
    if x=='control':
        pheno.append(0)
    if x=='type 2 diabetes':
        pheno.append(1)
#geno['AGE'] = list(dbt['Age'])       
geno['SEX'] = se
geno['PHENOTYPE'] = pheno
geno.head()

ID,rs3115849,rs4372192,rs6605066,rs3748592,rs3748595,rs3828047,rs3748596,rs3748597,rs4970441,rs13303229,rs6605071,rs7417106,rs13302983,rs13303355,rs6660139,rs15842,rs8997,rs3128098,rs2465128,rs3128102,rs10267,rs2710887,rs9803031,rs4074137,rs4562563,rs2887286,rs6603781,rs2748384,rs1570867,rs586965,rs1739855,rs2296474,rs1886773,rs307378,rs307377,rs307362,rs2765035,rs1892289,rs819976,rs819980,rs7533,rs6666293,rs61774918,rs2803296,rs2803348,rs941530,rs2803326,rs2803327,rs28548017,rs13303195,rs12184,rs2459986,rs2503701,rs2480714,rs1039100,rs2494598,rs2494632,rs2494631,rs12727342,rs10797428,rs2477699,rs2227313,rs4870,rs2257763,rs2234161,rs6667564,rs3748816,rs4648652,rs4648662,rs870124,rs2493323,rs2820999,rs2794322,rs2794340,rs10797402,rs10797403,rs2760321,rs4648409,rs6703035,rs2275832,rs4274008,rs1061968,rs3747989,rs11121915,rs2282281,rs875573,rs2229002,rs6696489,rs2843493,rs2273041,rs9435102,rs2294713,rs12735670,rs11590458,rs7548042,rs6693391,rs6693400,rs10864628,rs202069621,chr1_6640071_C_A,rs4243824,rs228669,rs228670,rs2859387,rs228696,rs1773135,rs172932,rs1058619,rs11121172,rs1058766,rs13596,rs2252865,rs3765968,rs9430181,rs6680090,rs1782455,rs2258621,rs1064261,rs2235663,rs2272983,rs6540995,rs6540996,rs6540997,rs4845880,rs4845881,rs6667720,rs6699881,rs4846044,rs11586659,rs868014,rs4846051,rs1994798,rs3795408,rs28551666,rs200753201,rs199733857,rs200378484,rs2487643,rs2486188,rs1046331,rs6429722,rs3753314,rs2473356,rs3737696,rs6429745,rs732286,rs945416,rs1572381,rs945417,rs1010069,rs1805152,rs2015509,rs6669935,rs10803407,rs11807054,rs3978493,rs2014562,rs1889789,rs1889790,rs7365182,rs7368151,rs11576236,rs2863456,rs221052,rs221024,rs3106498,rs10907226,rs1360575,rs557690,rs614461,rs2476568,rs2743505,rs9286084,rs2490829,rs2077142,rs872964,rs9435703,rs4920574,rs2746462,rs2647205,rs11203300,rs3003481,rs1635594,rs1621005,rs1635567,rs1204898,rs871088,rs871089,rs905389,rs2270977,rs2270976,rs1043459,rs2992753,rs3007718,rs2992752,rs9701796,rs28529092,rs2230708,rs7550822,rs7550938,rs2230706,rs2230705,rs941495,rs1043905,rs12584,rs1044010,rs2296108,rs3748759,rs3748757,rs951908,rs3762396,rs11588032,rs709683,rs710865,rs1738025,rs584367,rs6426616,rs4655194,rs2072753,rs230180,rs488991,rs537816,rs10799661,rs604659,rs604645,rs601329,rs592060,rs2230573,rs2305463,rs10916885,rs1395508,rs12024079,rs71512991,rs1780316,rs829368,rs829412,rs10799706,rs897471,rs989994,rs1874793,rs1874792,rs4654994,rs2229478,rs2501260,rs7528405,rs10733033,rs1803530,rs7534936,rs3820285,rs2501282,rs209729,rs172378,rs309472,rs681648,rs967605,rs10799790,rs1983965,rs605468,rs676363,rs1198423,rs1198430,rs11574,rs10917413,rs719400,rs2076344,rs146630384,rs6660949,rs10903083,rs4393101,rs7533060,rs11249170,rs946671,rs4090311,rs195718,rs195725,rs4649011,rs4649022,rs3093616,rs807270,rs807273,rs760598,rs760597,rs984412,rs6598924,rs2736831,rs2802321,rs2997452,rs2783680,rs10794531,rs6703878,rs7520030,rs11800553,rs921893,rs1466286,rs12723936,rs7543005,rs4970490,rs1064196,rs188158,rs282179,rs3170660,rs6689941,rs4266911,rs4418629,rs4908364,rs2076457,rs5018921,rs6697540,rs1042114,rs2234918,rs20566,rs1050663,rs6701521,rs10914377,rs10737358,rs10798884,rs3818788,rs2228550,rs552124,rs6664445,rs2762904,rs360042,rs360041,rs4951787,rs785274,rs604941,rs622407,rs4641276,rs6425816,rs4403594,rs9943259,rs2641959,rs1874044,rs519370,rs10737374,rs17342512,rs11578857,rs911218,rs10379,rs693582,rs2971408,rs3814302,rs595961,rs399393,rs11263871,rs7536394,rs2236565,rs368818766,rs215210,rs215211,rs2360010,rs6426024,rs684260,rs615530,rs4274021,rs12724533,rs7366048,rs4570384,rs2490948,rs3116396,rs2761764,rs7554206,rs784625,rs784634,rs2294814,rs3134614,rs491619,rs2011981,rs2011982,rs213768,rs2984691,rs9439043,rs2483689,rs615177,rs2810565,rs2810566,rs2984694,rs2475842,rs343376,rs1034268,rs3738494,rs6413971,rs2477561,rs2483688,rs3120276,rs539096,rs641365,rs641351,rs1143701,rs586339,rs2906465,rs3120803,rs2242637,rs803369,rs161721,rs3219493,rs3219487,rs3219468,rs2356559,rs3014211,rs2230657,rs2230658,rs2230659,rs1972410,rs299

In [12]:
print('Data size: ', geno.shape)

Data size:  (780, 11510)


In [13]:
# Split original data to training and testing data
X_train, X_test, y_train, y_test = train_test_split(geno.iloc[:,0:-1], geno.iloc[:,-1], test_size=0.2, random_state=42)
# Save the training and testing data
X_train.to_csv('../data/diabete_type2/train_test/X_train.csv')
X_test.to_csv('../data/diabete_type2/train_test/X_test.csv')
y_train.to_csv('../data/diabete_type2/train_test/y_train.csv')
y_test.to_csv('../data/diabete_type2/train_test/y_test.csv')
